In [1]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 

In [49]:
class MF(object):
    """docstring for CF"""
    
    def __init__(self, Y_data, K, lam = 0.1, Xinit = None, Winit = None, 
            learning_rate = 0.5, max_iter = 1000, print_every = 100, user_based = 1):
        self.Y_raw_data = Y_data
        self.K = K
        # regularization parameter
        self.lam = lam
        # learning rate for gradient descent
        self.learning_rate = learning_rate
        # maximum number of iterations
        self.max_iter = max_iter
        # print results after print_every iterations
        self.print_every = print_every
        # user-based or item-based
        self.user_based = user_based
        # number of users, items, and ratings. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(Y_data[:, 0])) + 1 
        self.n_items = int(np.max(Y_data[:, 1])) + 1
        self.n_ratings = Y_data.shape[0]
        
        if Xinit is None: # new
            self.X = np.random.randn(self.n_items, K)
        else: # or from saved data
            self.X = Xinit 
        
        if Winit is None: 
            self.W = np.random.randn(K, self.n_users)
        else: # from daved data
            self.W = Winit
            
        # normalized data, update later in normalized_Y function
        self.Y_data_n = self.Y_raw_data.copy()


    def normalize_Y(self):
        if self.user_based:
            user_col = 0
            item_col = 1
            n_objects = self.n_users

        # if we want to normalize based on item, just switch first two columns of data
        else: # item bas
            user_col = 1
            item_col = 0 
            n_objects = self.n_items

        users = self.Y_raw_data[:, user_col] 
        self.mu = np.zeros((n_objects,))
        for n in range(n_objects):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data_n[ids, item_col] 
            # and the corresponding ratings 
            ratings = self.Y_data_n[ids, 2]
            # take mean
            m = np.mean(ratings) 
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            self.mu[n] = m
            # normalize
            self.Y_data_n[ids, 2] = ratings - self.mu[n]
    
    #Loss function
    def loss(self):
        L = 0 
        for i in range(self.n_ratings):
            # user, item, rating
            n, m, rate = int(self.Y_data_n[i, 0]), int(self.Y_data_n[i, 1]), self.Y_data_n[i, 2]
            L += 0.5*(rate - self.X[m, :].dot(self.W[:, n]))**2
        
        # take average
        L /= self.n_ratings
        # regularization, don't ever forget this 
        L += 0.5*self.lam*(np.linalg.norm(self.X, 'fro') + np.linalg.norm(self.W, 'fro'))
        return L 
    
    def get_items_rated_by_user(self, user_id):
        """
        get all items which are rated by user user_id, and the corresponding ratings
        """
        ids = np.where(self.Y_data_n[:,0] == user_id)[0] 
        item_ids = self.Y_data_n[ids, 1].astype(np.int32) # indices need to be integers
        ratings = self.Y_data_n[ids, 2]
        return (item_ids, ratings)
    
    def get_users_who_rate_item(self, item_id):
        """
        get all users who rated item item_id and get the corresponding ratings
        """
        ids = np.where(self.Y_data_n[:,1] == item_id)[0] 
        user_ids = self.Y_data_n[ids, 0].astype(np.int32)
        ratings = self.Y_data_n[ids, 2]
        return (user_ids, ratings)
    
    def get_users_who_rate_item(self, item_id):
        """
        get all users who rated item item_id and get the corresponding ratings
        """
        ids = np.where(self.Y_data_n[:,1] == item_id)[0] 
        user_ids = self.Y_data_n[ids, 0].astype(np.int32)
        ratings = self.Y_data_n[ids, 2]
        return (user_ids, ratings)
    
    def updateX(self):
        for m in range(self.n_items):
            user_ids, ratings = self.get_users_who_rate_item(m)
            Wm = self.W[:, user_ids]
            # gradient
            grad_xm = -(ratings - self.X[m, :].dot(Wm)).dot(Wm.T)/self.n_ratings + \
                                               self.lam*self.X[m, :]
            self.X[m, :] -= self.learning_rate*grad_xm.reshape((self.K,))
            
            
    def updateW(self):
        for n in range(self.n_users):
            item_ids, ratings = self.get_items_rated_by_user(n)
            Xn = self.X[item_ids, :]
            # gradient
            grad_wn = -Xn.T.dot(ratings - Xn.dot(self.W[:, n]))/self.n_ratings + \
                        self.lam*self.W[:, n]
            self.W[:, n] -= self.learning_rate*grad_wn.reshape((self.K,))
            
            
    def fit(self):
        self.normalize_Y()
        for it in range(self.max_iter):
            self.updateX()
            self.updateW()
            if (it + 1) % self.print_every == 0:
                rmse_train = self.evaluate_RMSE(self.Y_raw_data)
                print('iter =', it + 1, ', loss =', self.loss(), ', RMSE train =', rmse_train)
                
                
    def pred(self, u, i):
        """ 
        predict the rating of user u for item i 
        if you need the un
        """
        u = int(u)
        i = int(i)
        if self.user_based:
            bias = self.mu[u]
        else: 
            bias = self.mu[i]
        pred = self.X[i, :].dot(self.W[:, u]) + bias 
        # truncate if results are out of range [0, 5]
        if pred < 0:
            return 0 
        if pred > 5: 
            return 5 
        return pred 
    
    
    def pred_for_user(self, user_id):
        """
        predict ratings one user give all unrated items
        """
        ids = np.where(self.Y_data_n[:, 0] == user_id)[0]
        items_rated_by_u = self.Y_data_n[ids, 1].tolist()              
        
        y_pred = self.X.dot(self.W[:, user_id]) + self.mu[user_id]
        predicted_ratings= []
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                predicted_ratings.append((i, y_pred[i]))
        
        return predicted_ratings
    
    
    def evaluate_RMSE(self, rate_test):
        n_tests = rate_test.shape[0]
        SE = 0 # squared error
        for n in range(n_tests):
            pred = self.pred(rate_test[n, 0], rate_test[n, 1])
            SE += (pred - rate_test[n, 2])**2 

        RMSE = np.sqrt(SE/n_tests)
        return RMSE
    
    
    

####  100k pandas

In [64]:
%%time
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('/home/suv/Documents/dask/dask/DASK/ml-100k/ub.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('/home/suv/Documents/dask/dask/DASK/ml-100k/ub.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base.as_matrix()
rate_test = ratings_test.as_matrix()

# indices start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

CPU times: user 21.7 ms, sys: 3.89 ms, total: 25.6 ms
Wall time: 29 ms


/home/suv/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/home/suv/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


####  Loss function 

In [54]:
%%time
rs = MF(rate_train, K = 10, lam = .1, print_every = 10, 
    learning_rate = 0.75, max_iter = 100, user_based = 1)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print('\nUser-based MF, RMSE =', RMSE)

iter = 10 , loss = 5.66490804581607 , RMSE train = 1.2138785915220693
iter = 20 , loss = 2.6406902841599447 , RMSE train = 1.037996330386298
iter = 30 , loss = 1.3438817776655667 , RMSE train = 1.0294649797021116
iter = 40 , loss = 0.7531095797655549 , RMSE train = 1.0291944732546747
iter = 50 , loss = 0.4823473200704482 , RMSE train = 1.029206577202196
iter = 60 , loss = 0.35818187795900575 , RMSE train = 1.0292124645525502
iter = 70 , loss = 0.301240230995628 , RMSE train = 1.0292139117325612
iter = 80 , loss = 0.2751272087571704 , RMSE train = 1.0292142382081932
iter = 90 , loss = 0.26315202415794137 , RMSE train = 1.0292143105552207
iter = 100 , loss = 0.2576603304302298 , RMSE train = 1.0292143265029872

User-based MF, RMSE = 1.060379895651066
CPU times: user 51.8 s, sys: 5.63 s, total: 57.4 s
Wall time: 50.5 s


In [55]:
%%time
rs = MF(rate_train, K = 10, lam = .1, print_every = 10, learning_rate = 0.75, max_iter = 100, user_based = 0)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print('\nItem-based MF, RMSE =', RMSE)

iter = 10 , loss = 5.647769502029098 , RMSE train = 1.1799037027250232
iter = 20 , loss = 2.626913557341967 , RMSE train = 1.0053051054108957
iter = 30 , loss = 1.328451550864264 , RMSE train = 0.996516290178449
iter = 40 , loss = 0.7368353199433277 , RMSE train = 0.9961795608274154
iter = 50 , loss = 0.46569451538745876 , RMSE train = 0.9961771125860625
iter = 60 , loss = 0.3413588150365551 , RMSE train = 0.9961798818375776
iter = 70 , loss = 0.2843400140642859 , RMSE train = 0.9961806643561328
iter = 80 , loss = 0.25819188275607446 , RMSE train = 0.9961808493230044
iter = 90 , loss = 0.2462006897841753 , RMSE train = 0.9961808915541871
iter = 100 , loss = 0.24070169022979812 , RMSE train = 0.9961809010941092

Item-based MF, RMSE = 1.0498047527866357
CPU times: user 54.8 s, sys: 4.86 s, total: 59.7 s
Wall time: 54 s


In [56]:
%%time
rs = MF(rate_train, K = 2, lam = 0, print_every = 10, learning_rate = 1, max_iter = 100, user_based = 0)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print('\nItem-based MF, RMSE =', RMSE)

iter = 10 , loss = 1.1843751235406905 , RMSE train = 1.491829838880295
iter = 20 , loss = 1.1177527136978844 , RMSE train = 1.4712797332515726
iter = 30 , loss = 1.0584884297666963 , RMSE train = 1.4518733108274333
iter = 40 , loss = 1.0055034074978622 , RMSE train = 1.4336306633980247
iter = 50 , loss = 0.9579127899928829 , RMSE train = 1.4164811861777777
iter = 60 , loss = 0.9149848679464789 , RMSE train = 1.400316052900905
iter = 70 , loss = 0.8761099965058639 , RMSE train = 1.3850088926114934
iter = 80 , loss = 0.8407766879071231 , RMSE train = 1.3705462731193565
iter = 90 , loss = 0.8085530360055262 , RMSE train = 1.3568984241935458
iter = 100 , loss = 0.7790721477313914 , RMSE train = 1.343997686418607

Item-based MF, RMSE = 1.4389155022656888
CPU times: user 51.8 s, sys: 3.9 s, total: 55.7 s
Wall time: 51 s


####  1M pandas

In [61]:
%%time 
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('/home/suv/Documents/dask/dask/DASK/ml-1m/ratings.dat', sep='::', names=r_cols, encoding='latin-1')

/home/suv/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


CPU times: user 4.34 s, sys: 125 ms, total: 4.46 s
Wall time: 4.39 s


####  20M pandas

In [68]:
%%time 
r_cols20 = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base20 = pd.read_csv('/home/suv/Documents/dask/dask/DASK/ml-20m/rating.csv')

CPU times: user 10.1 s, sys: 1.17 s, total: 11.3 s
Wall time: 11.3 s


In [67]:
ratings_base20

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24
